In [31]:
import yfinance as yf
import pandas as pd
import pandas as pd
import plotly.graph_objects as go
from datetime import datetime, timedelta
import numpy as np


In [32]:
date = '2024-04-12'

# Convert the date string to a datetime object
date_obj = datetime.strptime(date, '%Y-%m-%d')

# Check if the given date is a Monday
if date_obj.weekday() == 0:  # Monday is represented by 0
    # If it's Monday, subtract 3 days to get the previous working day
    date_previous = date_obj - timedelta(days=3)
else:
    # If it's not Monday, subtract 1 day to get the previous working day
    date_previous = date_obj - timedelta(days=1)

# Convert the datetime object back to a string in the same format
date_previous_str = date_previous.strftime('%Y-%m-%d')

In [33]:
## setup data source
trade = pd.read_csv(f'data/Performance/Performance_{date}.csv')
df = pd.read_csv(f'data/TradingData/nasdaq_100_5min_data_{date}.csv')
try:
    df_previous = pd.read_csv(f'data/TradingData/nasdaq_100_5min_data_{date_previous_str}.csv')
except FileNotFoundError as e:
    df_previous = df.copy()
    print(f"Cannot find file '{e.filename}'")


In [34]:
## Preprocess trade to trade_rth

trade = trade.drop(columns=['symbol','_priceFormat', '_priceFormatType', '_tickSize', 'buyFillId','sellFillId'])
# 1. Convert pnl to numeric format with proper handling of negative values
trade['pnl'] = trade['pnl'].str.replace('$', '')
trade['pnl'] = trade['pnl'].str.replace('(', '-').str.replace(')', '').astype(float)

# 2. Convert boughtTimestamp and soldTimestamp to datetime format and adjust to UTC
trade['boughtTimestamp'] = pd.to_datetime(trade['boughtTimestamp']) - pd.Timedelta(hours=7)
trade['soldTimestamp'] = pd.to_datetime(trade['soldTimestamp']) - pd.Timedelta(hours=7)

# 3. Convert duration to time format
trade['duration'] = pd.to_timedelta(trade['duration'])

#4. Filter out trades with boughtTimestamp or soldTimestamp between 9:30 and 16:00
trade_rth = trade[(trade['boughtTimestamp'].dt.time >= pd.Timestamp('09:30').time()) &
                       (trade['boughtTimestamp'].dt.time <= pd.Timestamp('16:00').time()) |
                       (trade['soldTimestamp'].dt.time >= pd.Timestamp('09:30').time()) &
                       (trade['soldTimestamp'].dt.time <= pd.Timestamp('16:00').time())].copy()  # Ensure a copy of the DataFrame is created

trade_rth['boughtTimestamp'] = trade_rth['boughtTimestamp'].dt.floor('min')
trade_rth['soldTimestamp'] = trade_rth['soldTimestamp'].dt.floor('min')

trade_rth['boughtTimestamp'] = trade_rth['boughtTimestamp'].dt.floor('5min')
trade_rth['soldTimestamp'] = trade_rth['soldTimestamp'].dt.floor('5min')


In [35]:
## Preprocess TradingData to RTH

# 0. Convert the 'Datetime' column to datetime dtype
df['Datetime'] = pd.to_datetime(df['Datetime'])
df.set_index('Datetime', inplace=True)

# 1. Convert the 'Datetime' column to datetime if it's not already in datetime format
df.index = pd.to_datetime(df.index)

# 2. Filter rows within the specified time range directly using the index
df_rth = df[(df.index.time >= pd.Timestamp('09:30').time()) & (df.index.time < pd.Timestamp('16:00').time())]

In [36]:
## Preprocess previous TradingData to df_previous_RTH

# 0. Convert the 'Datetime' column to datetime dtype
df_previous['Datetime'] = pd.to_datetime(df_previous['Datetime'])
df_previous.set_index('Datetime', inplace=True)

# 1. Convert the 'Datetime' column to datetime if it's not already in datetime format
df_previous.index = pd.to_datetime(df_previous.index)

# 2. Filter rows within the specified time range directly using the index
df_previous_rth = df_previous[(df_previous.index.time >= pd.Timestamp('09:30').time()) & (df_previous.index.time < pd.Timestamp('16:00').time())]


# 3. get previous open, close, highest and lowest
# Find the overall highest and lowest values for High and Low
pre_high = df_previous_rth['High'].max()
pre_low = df_previous_rth['Low'].min()

pre_open = df_previous_rth['Open'].iloc[0]
pre_close = df_previous_rth['Close'].iloc[-1]


# 4. Get trhe last 20 bars
df_previous_rth = df_previous_rth[-20:]

In [37]:
## Calculate EMA

# Combine previous and current trading session data
combined_df = pd.concat([df_previous_rth, df_rth])

# Calculate the 20-period EMA on the combined close prices
combined_df['EMA_20'] = combined_df['Close'].ewm(span=20, adjust=False).mean()


# Now, slice back out the EMA values that correspond only to df_rth dates
df_rth.loc[:, 'EMA_20'] = combined_df['EMA_20'].iloc[-len(df_rth):].values



/var/folders/cp/1g7t_8rd6kq_6f843vp6sg700000gn/T/ipykernel_5521/1545719625.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [38]:
import pandas as pd

# Assuming trade_rth is your DataFrame with trade data

# Calculate winning and losing trades
winning_trades = trade_rth[trade_rth['pnl'] > 0]
losing_trades = trade_rth[trade_rth['pnl'] <= 0]

# Data for pie chart
win_loss_counts = [len(winning_trades), len(losing_trades)]
labels = ['Winning Trades', 'Losing Trades']

# Generate statistics for tables
def trade_stats(trades):
    if len(trades) == 0:
        return {'Average PnL': 0, 'Max PnL': 0, 'Min PnL': 0, 'Total PnL': 0, 'Count': 0}
    return {
        'Average PnL': trades['pnl'].mean(),
        'Max PnL': trades['pnl'].max(),
        'Min PnL': trades['pnl'].min(),
        'Total PnL': trades['pnl'].sum(),
        'Count': len(trades)
    }

all_trades_stats = trade_stats(trade_rth)
winning_trades_stats = trade_stats(winning_trades)
losing_trades_stats = trade_stats(losing_trades)


In [39]:
import plotly.graph_objs as go

# Create pie chart trace for Plotly
pie_trace = go.Pie(labels=labels, values=win_loss_counts)

# Creating HTML for statistics tables
def generate_stats_table(stats, title):
    html_table = f"<h2>{title}</h2>"
    html_table += "<table border='1'><tr><th>Metric</th><th>Value</th></tr>"
    for key, value in stats.items():
        html_table += f"<tr><td>{key}</td><td>{value}</td></tr>"
    html_table += "</table>"
    return html_table

stats_tables_html = (
    generate_stats_table(all_trades_stats, "All Trades Statistics") +
    generate_stats_table(winning_trades_stats, "Winning Trades Statistics") +
    generate_stats_table(losing_trades_stats, "Losing Trades Statistics")
)


In [40]:

## Generate HTML 

plot_date = df_rth.index.date[0]  # Assuming df_rth is set up correctly

# Create a trace for the candlestick chart
trace_candlestick = go.Candlestick(
    x=df_rth.index.strftime('%Y-%m-%d %H:%M'),
    open=df_rth['Open'],
    high=df_rth['High'],
    low=df_rth['Low'],
    close=df_rth['Close']
)

# Create the pie chart trace
pie_trace = go.Pie(
    labels=['Winning Trades', 'Losing Trades'],
    values=win_loss_counts,
    name="Win vs Loss"
)

# Initialize a list for trade lines
trade_lines_and_points = []
for index, row in trade_rth.iterrows():
    entry_time, exit_time = sorted([row['boughtTimestamp'], row['soldTimestamp']])
    entry_price, exit_price = (row['buyPrice'], row['sellPrice']) if entry_time == row['boughtTimestamp'] else (row['sellPrice'], row['buyPrice'])
    color = 'green' if row['pnl'] > 0 else 'red'
    trade_line = go.Scatter(
        x=[entry_time, exit_time],
        y=[entry_price, exit_price],
        mode='lines+markers',
        line=dict(color=color, width=2),
        marker=dict(color=color, size=8),
        name=f"Trade {index}",
        hoverinfo='text',
        hovertext=f"Entry: {entry_time}, Price: {entry_price} | Exit: {exit_time}, Price: {exit_price}, PnL: {row['pnl']}"
    )
    trade_lines_and_points.append(trade_line)
    
# Create a trace for the EMA line
trace_ema = go.Scatter(
    x=df_rth.index.strftime('%Y-%m-%d %H:%M'),
    y=df_rth['EMA_20'],
    mode='lines',  # Only lines, no markers
    line=dict(color='blue', width=2),  # Customize line color and width
    name='EMA_20'  # Legend label
)

# Generate horizontal lines
horizontal_lines = [
    go.Scatter(
        x=[df_rth.index.min(), df_rth.index.max()],
        y=[value] * 2,
        mode='lines',
        line=dict(color=color, width=2, dash='dash'),
        name=f'pre_{label.lower()}'  # Constructing the name dynamically
    ) for value, color, label in zip([pre_open, pre_close, pre_high, pre_low], ['orange', 'purple', 'green', 'red'], ['Open', 'Close', 'High', 'Low'])
]


# Generate annotations for each bar
low_points = df_rth['Low']
offset = low_points.min() * 0.001
annotations = [{
    'x': date,
    'y': low - offset,
    'xref': 'x',
    'yref': 'y',
    'text': str(i),
    'showarrow': False,
    'font': {'family': 'Arial, sans-serif', 'size': 12, 'color': 'black'},
    'align': 'center'
} for i, (date, low) in enumerate(zip(df_rth.index, low_points), start=1)]

# Define layout
layout = go.Layout(
    title=f'NASDAQ 100 Futures Candlestick Chart - {plot_date}',
    xaxis_title='Datetime',
    yaxis_title='Price',
    xaxis_rangeslider_visible=False,
    xaxis=dict(tickangle=-45),
    yaxis=dict(tickformat="none"),
    annotations=annotations
)

# Create figure and add traces

fig = go.Figure(data=[trace_candlestick] + trade_lines_and_points + [trace_ema], layout=layout)
# Add the horizontal lines to the figure
fig.add_traces(horizontal_lines)

# Define the JavaScript for resizing
resize_script = """
<script>
function resizePlot() {
    var plotElement = document.getElementById('plot');
    plotElement.style.width = window.innerWidth * 0.8 + 'px';
    plotElement.style.height = window.innerHeight * 0.8 + 'px';
    Plotly.relayout(plotElement, {
        width: window.innerWidth * 0.8,
        height: window.innerHeight * 0.8
    });
}
window.onresize = resizePlot;
window.onload = resizePlot;
</script>
"""

# Generate the full HTML content including your resizing script
full_html = f"""
<html>
<head>
    <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
</head>
<body>
    <div id="plot" style="width: 100%; height: 100%;"></div>
    {resize_script}
    <script>
        var plotElement = document.getElementById('plot');
        var data = {fig.to_json()};
        Plotly.newPlot(plotElement, data.data, data.layout);
    </script>
</body>
</html>
"""

# Append the pie chart to the existing figure (or it can be a separate div for separate control)
fig.add_trace(pie_trace)

# Additional HTML and JS for pie chart and tables
additional_html = f"""
<div style='display: flex; justify-content: space-around;'>
    <div id='pieChart' style='width: 400px; height: 400px;'></div>
    <div id='tradeStats'>
        {stats_tables_html}
    </div>
</div>
<script>
    var pieData = [{pie_trace.to_json()}];
    Plotly.newPlot('pieChart', pieData);
</script>
"""

# Append to your existing full_html variable
full_html += additional_html

# Save the customized HTML file

# Define the folder path
folder_path = "./data//HTML/"

# Save the data to a CSV file with the date in the filename within the specified folder
file_name = folder_path + f"NASDAQ_100_Futures_Candlestick_Chart_{plot_date}.html"


html_file_name = file_name
with open(html_file_name, "w") as f:
    f.write(full_html)
